# Perkins V - Calculating Performance Indicators by CIP

In [37]:
# Author: Matthew Fikes
# Modified: 3/11/22
import pandas as pd


### This code will calculate the Perkins 2P1 and 3P1 performance indicators.

#### If your file is not named CTEA.xlsx the code will prompt you to enter the filename. This assumes values in CTEA file are numeric and not descriptions (e.g. non-completion status codes are 4 and 6). This was designed only for the CTEA-1 but can be modified.

In [2]:
file = 'CTEA.xlsx'

try:
    data = pd.read_excel(file)
    print('File {0} loaded successfully'.format(file))
except:
    file = input('CTEA.xlsx not found. Please enter the full filename: ')
    try:
        data = pd.read_excel(file)
        print('File {0} loaded successfully'.format(file))
    except:
        print('No file found')
        exit
    

File CTEA.xlsx loaded successfully


## Static Variables
These can be modified to suit your individual file. Targets for 1P1, 2P1, and 3P1 ar also included so they can be easily changed.

In [3]:
target_1 = .4975
target_2 = .2075
target_3 = .1675

In [4]:
# fields to use for calculations from merged CTEA 1A/B - can be renamed if your fields are different
cip_field = 'CTEA 1A / CIP Code / DYN||8774_DYN'
status_field = 'CTEA 1A / YRENDSTAT_ID / DYN||8765_DYN'
credit_field = 'CTEA 1A / Credits Earned / DYN||8763_DYN'
emp_field = 'CTEA 1B / EMPSTAT_ID / DYN||8785_DYN'
educ_field = 'CTEA 1B / EDUCSTAT_ID / DYN||8784_DYN'
id_field = 'CTEA 1A / Student ID / DYN||8761_DYN'
gender_field = 'CTEA 1A / Gender / DYN||8770_DYN'

In [5]:
# special population fields
disab_field = 'CTEA 1A / Disabled / DYN||8766_DYN'
displ_field = 'CTEA 1A / DISPLACED_HM / DYN||8776_DYN'
econdis_field = 'CTEA 1A / Economic Disadvantage / DYN||8777_DYN'
homels_field = 'CTEA 1A / Homeless / DYN||8769_DYN'
lim_eng_field = 'CTEA 1A / Limited English / DYN||8780_DYN'
migr_field = 'CTEA 1A / Migrant / DYN||8772_DYN'
sparnt_field = 'CTEA 1A / Single Parent / DYN||8775_DYN'
youth_ao_field = 'CTEA 1A / Youth Aged Out / DYN||8779_DYN'
youth_af_field = 'CTEA 1A / Youth Armed Forces / DYN||8762_DYN'
spop_fields = [disab_field,displ_field,econdis_field,homels_field,lim_eng_field,migr_field,sparnt_field,youth_ao_field,youth_af_field]

### Get Non-traditional Crosswalk

In [6]:
xwalk_url = 'https://s3.amazonaws.com/PCRN/docs/REVISED_FINAL-2020-Nontraditional-Crosswalk-6-9-2021.xlsx'
try:
    print('Downloading Non-traditional crosswalk from ',xwalk_url)
    nontrad_xwalk = pd.read_excel(xwalk_url)
    print('Loaded successfully')
except:
    print('Unable to connect to ',xwalk_url)

Loaded successfully


### Join CTEA to crosswalk and separate completers

In [7]:
# merge crosswalk with loaded CTEA file, joined by CIP
new_data = nontrad_xwalk[['CIP 6 2020','Female','Male']].merge(data,left_on='CIP 6 2020',right_on=cip_field,how='right')

In [8]:
# splits completers based on status codes 4 and 6 and total credits >=12
completer_set = new_data[(new_data[status_field].isin([4,6])) | (new_data[credit_field]>=12)] #make numerator data

In [9]:
# uncomment to make dataframe of noncompleters
#non_completers = pd.merge(new_data,completer_set,how='outer',on=id_field,indicator=True)
#noncomp_df = non_completers.loc[non_completers._merge == 'left_only']

# 1P1 Performance Indicator

In [10]:
num_1p1 = completer_set[(completer_set[status_field].isin([4,6])) & 
                        ((completer_set[emp_field].isin([1,2,3,7,8,9]))|
                          (completer_set[educ_field]==1))]
den_1p1 = completer_set[(completer_set[status_field].isin([4,6]))]

In [11]:
num_1p1

,CIP 6 2020,Female,Male,INST_ID / BEDSCODE,CTEA 1A / Student ID / DYN||8761_DYN,CTEA 1A / IRP Code / DYN||8771_DYN,CTEA 1A / CIP Code / DYN||8774_DYN,CTEA 1A / Gender / DYN||8770_DYN,CTEA 1A / Ethnicity / DYN||8781_DYN_MA,CTEA 1A / Non Resident Alien / DYN||8767_DYN,...,CTEA 1A / Credits Earned / DYN||8763_DYN,CTEA 1A / YRENDSTAT_ID / DYN||8765_DYN,CTEA 1A / AWIRP Code / DYN||8778_DYN,CTEA 1A / TSA Taken / DYN||8764_DYN,CTEA 1A / TSA Passed / DYN||8773_DYN,CTEA 1B / EMPSTAT_ID / DYN||8785_DYN,CTEA 1B / EDUCSTAT_ID / DYN||8784_DYN,CTEA 1B / Surveyed / DYN||8786_DYN,CTEA 1B / Responded to Survey / DYN||8783_DYN,Nontrad
2,15.0201,Y,N,800000041234,P00414694,1170,15.0201,1,6,2,...,102.0,4,1170.0,0,0,6,1,2,2,F
5,15.0201,Y,N,800000041234,P00416589,1170,15.0201,1,6,2,...,73.0,4,1170.0,0,0,2,3,2,2,F
10,15.0201,Y,N,800000041234,P00404171,1170,15.0201,1,6,2,...,86.0,4,1170.0,0,0,1,3,2,2,F
11,15.0201,Y,N,800000041234,P00426804,1170,15.0201,1,6,2,...,75.0,4,1170.0,0,0,6,1,2,2,F
14,15.0201,Y,N,800000041234,P00232111,1170,15.0201,1,6,2,...,156.5,4,1170.0,0,0,2,3,2,2,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369,48.0510,Y,N,800000041234,P00410086,22622,48.0510,1,6,2,...,105.0,4,22622.0,0,0,2,3,2,2,F
2385,48.0510,Y,N,800000041234,P00427551,22622,48.0510,1,6,2,...,35.0,6,22622.0,0,0,1,3,2,2,F
2388,48.0510,Y,N,800000041234,P00419902,22622,48.0510,1,6,2,...,96.0,4,22622.0,0,0,1,3,2,2,F
2390,48.0510,Y,N,800000041234,P00419902,22622,48.0510,1,6,2,...,96.0,4,22622.0,0,0,1,3,2,2,F


In [12]:
# Calculate performance for each special population by CIP, return values below target
rows_1p1 = []
for i in spop_fields:

    lim_num = pd.DataFrame(num_1p1[(num_1p1[i]==1)])
    lim_den = pd.DataFrame(den_1p1[(den_1p1[i]==1)])

    num = pd.DataFrame(lim_num).filter([cip_field,i]).groupby(cip_field).count()
    dem = pd.DataFrame(lim_den).filter([cip_field,i]).groupby(cip_field).count()
    pct = num.divide(dem)
    underperf = pct[(pct.values<=target_1)]
    
    rows_1p1.append(underperf.reset_index())
    
spop_1p1 = pd.concat(rows_1p1).dropna(axis=1,how='all')

In [13]:
num_grp_1 = num_1p1.groupby(cip_field)[id_field].count()
den_grp_1 = den_1p1.groupby(cip_field)[id_field].count()
percents_1p1 = num_grp_1.divide(den_grp_1,fill_value=0)

## MOST RECENT PERFORMANCE DATA BY PROGRAM

In [14]:
pd.DataFrame(percents_1p1).reset_index().set_index('CTEA 1A / CIP Code / DYN||8774_DYN')

,CTEA 1A / Student ID / DYN||8761_DYN
CTEA 1A / CIP Code / DYN||8774_DYN,
11.0201,0.333333
11.0401,0.800000
11.0701,0.666667
11.1003,0.750000
15.0201,0.888889
15.0303,0.666667
15.0407,0.000000
15.0801,1.000000
15.0805,0.333333


## PROGRAMS NOT MEETING TARGET

In [15]:

p_unmet_1p1 = pd.DataFrame(percents_1p1[(percents_1p1.values<=target_1)]).reset_index().set_index('CTEA 1A / CIP Code / DYN||8774_DYN')

In [16]:
p_unmet_1p1

,CTEA 1A / Student ID / DYN||8761_DYN
CTEA 1A / CIP Code / DYN||8774_DYN,
11.0201,0.333333
15.0407,0.000000
15.0805,0.333333
43.0103,0.366667
43.0107,0.312500
47.0201,0.000000
48.0508,0.369565
50.0102,0.230769
50.0410,0.210526


## SPECIAL POPULATIONS NOT MEETING TARGET WITHIN PROGRAMS

In [17]:
# Special Pops falling below 1P1 target, by CIP
unmet_spop_1p1 = spop_1p1.set_index('CTEA 1A / CIP Code / DYN||8774_DYN')
unmet_spop_1p1

,CTEA 1A / Disabled / DYN||8766_DYN,CTEA 1A / DISPLACED_HM / DYN||8776_DYN,CTEA 1A / Economic Disadvantage / DYN||8777_DYN,CTEA 1A / Limited English / DYN||8780_DYN,CTEA 1A / Single Parent / DYN||8775_DYN
CTEA 1A / CIP Code / DYN||8774_DYN,,,,,
50.0102,0.400000,NaN,NaN,NaN,NaN
50.0410,0.333333,NaN,NaN,NaN,NaN
52.0201,0.333333,NaN,NaN,NaN,NaN
51.3801,NaN,0.333333,NaN,NaN,NaN
15.0805,NaN,NaN,0.250000,NaN,NaN
43.0103,NaN,NaN,0.375000,NaN,NaN
43.0107,NaN,NaN,0.357143,NaN,NaN
48.0508,NaN,NaN,0.333333,NaN,NaN
50.0102,NaN,NaN,0.200000,NaN,NaN


# 2P1 Performance Indicator

In [18]:

num_2p1 = completer_set[(completer_set[status_field].isin([4,6]))]
den_2p1 = completer_set[(completer_set[status_field].isin([4,5,6]))]

In [19]:
num_grp_2 = num_2p1.groupby(cip_field)[id_field].count()

den_grp_2 = den_2p1.groupby(cip_field)[id_field].count()

In [20]:
# Calculate performance for each special population by CIP, return values below target
rows_2p1 = []
for i in spop_fields:

    lim_num = pd.DataFrame(num_2p1[(num_2p1[i]==1)])
    lim_den = pd.DataFrame(den_2p1[(den_2p1[i]==1)])

    num = pd.DataFrame(lim_num).filter([cip_field,i]).groupby(cip_field).count()
    dem = pd.DataFrame(lim_den).filter([cip_field,i]).groupby(cip_field).count()
    pct = num.divide(dem)
    underperf = pct[(pct.values<=target_2)]
    
    rows_2p1.append(underperf.reset_index())
    
spop_2p1 = pd.concat(rows_2p1).dropna(axis=1,how='all')

In [21]:
spop_2p1

,CTEA 1A / CIP Code / DYN||8774_DYN,CTEA 1A / Disabled / DYN||8766_DYN,CTEA 1A / Economic Disadvantage / DYN||8777_DYN,CTEA 1A / Single Parent / DYN||8775_DYN
0,51.0000,0.2,NaN,NaN
0,51.0000,NaN,0.08,NaN
0,43.0103,NaN,NaN,0.2


In [22]:
percents_2p1 = num_grp_2.divide(den_grp_2,fill_value=0)


## MOST RECENT PERFORMANCE DATA BY PROGRAM

In [23]:
pd.DataFrame(percents_2p1).reset_index().set_index('CTEA 1A / CIP Code / DYN||8774_DYN')

,CTEA 1A / Student ID / DYN||8761_DYN
CTEA 1A / CIP Code / DYN||8774_DYN,
11.0201,0.666667
11.0401,0.454545
11.0701,0.545455
11.1003,0.727273
15.0201,0.900000
15.0303,0.461538
15.0407,0.666667
15.0616,0.000000
15.0801,0.800000


## PROGRAMS NOT MEETING TARGET

In [24]:

p_unmet_2p1 = pd.DataFrame(percents_2p1[(percents_2p1.values<=target_2)]).reset_index().set_index('CTEA 1A / CIP Code / DYN||8774_DYN')

In [25]:
p_unmet_2p1

,CTEA 1A / Student ID / DYN||8761_DYN
CTEA 1A / CIP Code / DYN||8774_DYN,
15.0616,0.000000
51.0000,0.103448
52.0703,0.000000


## SPECIAL POPULATIONS NOT MEETING TARGET WITHIN PROGRAMS

In [26]:
# Special Pops falling below 1P1 target, by CIP
unmet_spop_2p1 = spop_2p1.set_index('CTEA 1A / CIP Code / DYN||8774_DYN')
unmet_spop_2p1

,CTEA 1A / Disabled / DYN||8766_DYN,CTEA 1A / Economic Disadvantage / DYN||8777_DYN,CTEA 1A / Single Parent / DYN||8775_DYN
CTEA 1A / CIP Code / DYN||8774_DYN,,,
51.0000,0.2,NaN,NaN
51.0000,NaN,0.08,NaN
43.0103,NaN,NaN,0.2


# 3P1 Performance Indicator

In [27]:
num_3p1 = completer_set[((completer_set['Female']=='Y') & (completer_set[gender_field]==2))|((completer_set['Male']=='Y') & (completer_set[gender_field]==1))]
num_grp_3 = num_3p1.groupby(cip_field)[id_field].count()

den_3p1 = completer_set[(completer_set['Female']=='Y') | (completer_set['Male']=='Y')]
den_grp_3 = den_3p1.groupby(cip_field)[id_field].count()

percents_3p1 = num_grp_3.divide(den_grp_3,fill_value=0)


In [28]:
# Calculate performance for each special population by CIP, return values below target
rows_3p1 = []
for i in spop_fields:

    lim_num = pd.DataFrame(num_3p1[(num_3p1[i]==1)])
    lim_den = pd.DataFrame(den_3p1[(den_3p1[i]==1)])

    num = pd.DataFrame(lim_num).filter([cip_field,i]).groupby(cip_field).count()
    dem = pd.DataFrame(lim_den).filter([cip_field,i]).groupby(cip_field).count()
    pct = num.divide(dem)
    underperf = pct[(pct.values<=target_3)]
    
    rows_3p1.append(underperf.reset_index())
    
spop_3p1 = pd.concat(rows_3p1).dropna(axis=1,how='all')

## MOST RECENT PERFORMANCE DATA BY PROGRAM

In [30]:
pd.DataFrame(percents_3p1).reset_index().set_index('CTEA 1A / CIP Code / DYN||8774_DYN')

,CTEA 1A / Student ID / DYN||8761_DYN
CTEA 1A / CIP Code / DYN||8774_DYN,
11.0201,0.131579
11.0401,0.285714
11.0701,0.434783
11.1003,0.226415
13.1314,0.000000
15.0201,0.033333
15.0303,0.148148
15.0407,0.000000
15.0616,0.000000


## PROGRAMS NOT MEETING TARGET

In [31]:

p_unmet_3p1 = pd.DataFrame(percents_3p1[(percents_3p1.values<=target_3)]).reset_index().set_index('CTEA 1A / CIP Code / DYN||8774_DYN')
p_unmet_3p1

,CTEA 1A / Student ID / DYN||8761_DYN
CTEA 1A / CIP Code / DYN||8774_DYN,
11.0201,0.131579
13.1314,0.000000
15.0201,0.033333
15.0303,0.148148
15.0407,0.000000
15.0616,0.000000
15.0801,0.000000
15.0805,0.057143
15.1302,0.100000


## SPECIAL POPULATIONS NOT MEETING TARGET WITHIN PROGRAMS

In [32]:
# Special Pops falling below 1P1 target, by CIP
unmet_spop_3p1 = spop_3p1.set_index('CTEA 1A / CIP Code / DYN||8774_DYN')
unmet_spop_3p1

,CTEA 1A / Disabled / DYN||8766_DYN,CTEA 1A / Economic Disadvantage / DYN||8777_DYN,CTEA 1A / Single Parent / DYN||8775_DYN
CTEA 1A / CIP Code / DYN||8774_DYN,,,
48.0508,0.1,NaN,NaN
11.0201,NaN,0.142857,NaN
15.0201,NaN,0.040000,NaN
15.0303,NaN,0.125000,NaN
47.0201,NaN,0.031250,NaN
51.3801,NaN,0.159091,NaN
52.0205,NaN,0.150000,NaN
51.3801,NaN,NaN,0.105263
52.0302,NaN,NaN,0.111111


## RESULTS

In [33]:
# load measures into dataframe
df1a = pd.DataFrame(percents_1p1).reset_index()
df1a['Target 1'] = target_1
df1a.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Measure 1P1'},inplace=True)
df1b = df1a.merge(p_unmet_1p1,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')
df1b.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Unmet 1P1'},inplace=True)
df1c =spop_1p1
df1c.rename(columns={'CTEA 1A / Disabled / DYN||8766_DYN':'Unmet 1P1 Disabled',
                     'CTEA 1A / DISPLACED_HM / DYN||8776_DYN': 'Unmet 1P1 Displaced',
                     'CTEA 1A / Economic Disadvantage / DYN||8777_DYN': 'Unmet 1P1 Econ Dis',
                     'CTEA 1A / Homeless / DYN||8769_DYN': 'Unmet 1P1 Homeless',
                     'CTEA 1A / Limited English / DYN||8780_DYN': 'Unmet 1P1 Lim Eng',
                     'CTEA 1A / Migrant / DYN||8772_DYN': 'Unmet 1P1 Migrant',
                     'CTEA 1A / Single Parent / DYN||8775_DYN': 'Unmet 1P1 Sing Par',
                     'CTEA 1A / Youth Aged Out / DYN||8779_DYN': 'Unmet 1P1 Aged Out',
                     'CTEA 1A / Youth Armed Forces / DYN||8762_DYN': 'Unmet 1P1 Armed Forces'
                     
                    },inplace=True)
df1 = df1b.merge(df1c,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')



df2a = pd.DataFrame(percents_2p1).reset_index()
df2a['Target 2'] = target_2
df2a.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Measure 2P1'},inplace=True)
df2b = df2a.merge(p_unmet_2p1,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')
df2b.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Unmet 2P1'},inplace=True)
df2c =spop_2p1
df2c.rename(columns={'CTEA 1A / Disabled / DYN||8766_DYN':'Unmet 2P1 Disabled',
                     'CTEA 1A / DISPLACED_HM / DYN||8776_DYN': 'Unmet 2P1 Displaced',
                     'CTEA 1A / Economic Disadvantage / DYN||8777_DYN': 'Unmet 2P1 Econ Dis',
                     'CTEA 1A / Homeless / DYN||8769_DYN': 'Unmet 2P1 Homeless',
                     'CTEA 1A / Limited English / DYN||8780_DYN': 'Unmet 2P1 Lim Eng',
                     'CTEA 1A / Migrant / DYN||8772_DYN': 'Unmet 2P1 Migrant',
                     'CTEA 1A / Single Parent / DYN||8775_DYN': 'Unmet 2P1 Sing Par',
                     'CTEA 1A / Youth Aged Out / DYN||8779_DYN': 'Unmet 2P1 Aged Out',
                     'CTEA 1A / Youth Armed Forces / DYN||8762_DYN': 'Unmet 2P1 Armed Forces'
                     
                    },inplace=True)
df2 = df2b.merge(df2c,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')


df3a = pd.DataFrame(percents_3p1).reset_index()
df3a['Target 3'] = target_3
df3a.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Measure 3P1'},inplace=True)
df3b = df3a.merge(p_unmet_3p1,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')
df3b.rename(columns={'CTEA 1A / Student ID / DYN||8761_DYN':'Unmet 3P1'},inplace=True)
df3c =spop_3p1
df3c.rename(columns={'CTEA 1A / Disabled / DYN||8766_DYN':'Unmet 3P1 Disabled',
                     'CTEA 1A / DISPLACED_HM / DYN||8776_DYN': 'Unmet 3P1 Displaced',
                     'CTEA 1A / Economic Disadvantage / DYN||8777_DYN': 'Unmet 3P1 Econ Dis',
                     'CTEA 1A / Homeless / DYN||8769_DYN': 'Unmet 3P1 Homeless',
                     'CTEA 1A / Limited English / DYN||8780_DYN': 'Unmet 3P1 Lim Eng',
                     'CTEA 1A / Migrant / DYN||8772_DYN': 'Unmet 3P1 Migrant',
                     'CTEA 1A / Single Parent / DYN||8775_DYN': 'Unmet 3P1 Sing Par',
                     'CTEA 1A / Youth Aged Out / DYN||8779_DYN': 'Unmet 3P1 Aged Out',
                     'CTEA 1A / Youth Armed Forces / DYN||8762_DYN': 'Unmet 3P1 Armed Forces'
                     
                    },inplace=True)
df3 = df3b.merge(df3c,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')

m1 = df1.merge(df2,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')
measures = m1.merge(df3,on='CTEA 1A / CIP Code / DYN||8774_DYN',how='left')
measures.rename(columns={'CTEA 1A / CIP Code / DYN||8774_DYN':'CIP Code'},inplace=True)
measures['CIP Code'] = measures['CIP Code'].map('{:.4f}'.format)


In [34]:
measures

,CIP Code,Measure 1P1,Target 1,Unmet 1P1,Unmet 1P1 Disabled,Unmet 1P1 Displaced,Unmet 1P1 Econ Dis,Unmet 1P1 Lim Eng,Unmet 1P1 Sing Par,Measure 2P1,...,Unmet 2P1,Unmet 2P1 Disabled,Unmet 2P1 Econ Dis,Unmet 2P1 Sing Par,Measure 3P1,Target 3,Unmet 3P1,Unmet 3P1 Disabled,Unmet 3P1 Econ Dis,Unmet 3P1 Sing Par
0,11.0201,0.333333,0.4975,0.333333,NaN,NaN,NaN,NaN,NaN,0.666667,...,NaN,NaN,NaN,NaN,0.131579,0.1675,0.131579,NaN,0.142857,NaN
1,11.0401,0.800000,0.4975,NaN,NaN,NaN,NaN,NaN,NaN,0.454545,...,NaN,NaN,NaN,NaN,0.285714,0.1675,NaN,NaN,NaN,NaN
2,11.0701,0.666667,0.4975,NaN,NaN,NaN,NaN,NaN,NaN,0.545455,...,NaN,NaN,NaN,NaN,0.434783,0.1675,NaN,NaN,NaN,NaN
3,11.1003,0.750000,0.4975,NaN,NaN,NaN,NaN,NaN,NaN,0.727273,...,NaN,NaN,NaN,NaN,0.226415,0.1675,NaN,NaN,NaN,NaN
4,15.0201,0.888889,0.4975,NaN,NaN,NaN,NaN,NaN,NaN,0.900000,...,NaN,NaN,NaN,NaN,0.033333,0.1675,0.033333,NaN,0.040000,NaN
5,15.0303,0.666667,0.4975,NaN,NaN,NaN,NaN,NaN,NaN,0.461538,...,NaN,NaN,NaN,NaN,0.148148,0.1675,0.148148,NaN,0.125000,NaN
6,15.0407,0.000000,0.4975,0.000000,NaN,NaN,NaN,NaN,NaN,0.666667,...,NaN,NaN,NaN,NaN,0.000000,0.1675,0.000000,NaN,NaN,NaN
7,15.0801,1.000000,0.4975,NaN,NaN,NaN,NaN,NaN,NaN,0.800000,...,NaN,NaN,NaN,NaN,0.000000,0.1675,0.000000,NaN,NaN,NaN
8,15.0805,0.333333,0.4975,0.333333,NaN,NaN,0.250000,NaN,NaN,0.461538,...,NaN,NaN,NaN,NaN,0.057143,0.1675,0.057143,NaN,NaN,NaN
9,15.1302,0.500000,0.4975,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,...,NaN,NaN,NaN,NaN,0.100000,0.1675,0.100000,NaN,NaN,NaN


In [35]:
print('Overall Performance for 1P1 is {0:.2f}%'.format(len(num_1p1)/len(den_1p1)*100))
print('Overall Performance for 2P1 is {0:.2f}%'.format(len(num_2p1)/len(den_2p1)*100))
print('Overall Performance for for 3P1 is {0:.2f}%'.format(len(num_3p1)/len(den_3p1)*100))

Overall Performance for 1P1 is 52.44%
Overall Performance for 2P1 is 69.35%
Overall Performance for for 3P1 is 27.19%


In [ ]:
    
print(measures)
choice = input('Export measures to Excel? Y/N:')
if ((choice =='Y') | (choice =='y')):
    measures.to_excel('measures.xlsx',index=None)
    print('Exported to measures.xlsx')
else:
    
    pass